# Work pool, worker management in Prefect

## 1. Introduction of Work pool and worker

`Worker and Wook Pool` are the most fundamental architectural concepts in `Prefect 3`.

### 1.1 Work pool

In `Prefect 3`, a **Work Pool** represents A `logical namespace` in the Prefect Server. It defines infrastructure type (e.g. process, docker, kubernetes, etc.) and global policies like `concurrency limit`.

> It can hold one or more `work queues` that describe job priorities. I don't think the `work queues` are very useful for Constances.

### 1.2 Worker

The actual `task execution agent` (e.g. process, container, etc.) of the workflow. It connects to **exactly one work pool**. It continuously polls scheduled flows from the connected work pool and executes each flow with the provided environment(i.e. python virtual env).


### 1.3 Worker VS Work Pool

A `worker` must belong to one and only one `work pool`.

A `work pool`:
- can have one or multiple attached `workers`.
- can have one or multiple `work queue`

Below is an example. A work-pool has three work queues, and three workers are attached to the work-pool

```text
(pliu-pool)
 ├── queue: daily-etl
 ├── queue: export
 └── queue: validation
        ▲
        │
 ┌────────────────────────────────────┐
 │  Worker A (process type, limit=1) │
 │  Worker B (process type, limit=1) │
 │  Worker C (process type, limit=1) │
 └────────────────────────────────────┘
```

## Work pool management
We can manage the `Prefect work pools` by using the `web UI, CLI, Terraform, or REST API`. In this tutorial, I will only show how to do it via the `CLI`. Because it's the best way to share and document how to manage work pool.

Below shows a CLI cheat list

```powershell
# The general form of the CLI for managing work pool is
prefect work-pool [OPTIONS] COMMAND [ARGS]...

# you can get all possible commands with
prefect work-pool --help


# show exiting work pool
prefect work-pool ls

# you should see something like
                                    Work Pools
┏━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Name       ┃ Type    ┃                                   ID ┃ Concurrency Limit ┃
┡━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ pliu-pool  │ process │ 35ed8ee0-dbe7-424b-91ad-b46f49395f54 │ None              │
│ user3_pool │ process │ 26865923-476b-4fc6-a9fa-3aaf6347b5e2 │ None              │
│ user2_pool │ process │ 2bd75b5f-b5fb-4f82-a940-6db17d09c348 │ None              │
│ user1_pool │ process │ 0957b871-0554-4c04-95b2-c0a3ead78b8e │ None              │
│ local-pool │ process │ 05494dd0-20ce-4116-8bf9-5f557549d7ac │ None              │
└────────────┴─────────┴──────────────────────────────────────┴───────────────────┘

# inspect a work pool
prefect work-pool inspect 'pliu-pool'

# you should see something like
WorkPool(
    id='35ed8ee0-dbe7-424b-91ad-b46f49395f54',
    created=DateTime(2025, 10, 29, 8, 49, 41, 890635, tzinfo=Timezone('UTC')),
    updated=DateTime(2025, 10, 29, 14, 29, 58, 925000, tzinfo=Timezone('UTC')),
    name='pliu-pool',
    type='process',
    base_job_template={
        'job_configuration': {'command': '{{ command }}', 'env': '{{ env }}', 'labels': '{{ labels }}', 'name': '{{ name }}', 'stream_output': '{{ stream_output }}', 'working_dir': '{{ working_dir }}'},
        'variables': {
            'properties': {
                'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Name given to infrastructure created by a worker.', 'title': 'Name'},
                'env': {'additionalProperties': {'anyOf': [{'type': 'string'}, {'type': 'null'}]}, 'description': 'Environment variables to set when starting a flow run.', 'title': 'Environment Variables', 'type': 'object'},
                'labels': {'additionalProperties': {'type': 'string'}, 'description': 'Labels applied to infrastructure created by a worker.', 'title': 'Labels', 'type': 'object'},
                'command': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'The command to use when starting a flow run. In most cases, this should be left blank and the command will be automatically generated by the worker.', 'title': 'Command'},
                'stream_output': {'default': True, 'description': 'If enabled, workers will stream output from flow run processes to local standard output.', 'title': 'Stream Output', 'type': 'boolean'},
                'working_dir': {
                    'anyOf': [{'format': 'path', 'type': 'string'}, {'type': 'null'}],
                    'default': None,
                    'description': 'If provided, workers will open flow run processes within the specified path as the working directory. Otherwise, a temporary directory will be created.',
                    'title': 'Working Directory'
                }
            },
            'type': 'object'
        }
    },
    status=WorkPoolStatus.READY,
    storage_configuration=WorkPoolStorageConfiguration(),
    default_queue_id='2b7c59fb-39f1-4c24-8821-b99e0663798a'
)

# Preview scheduled work of a work pool in the next 12 hours
prefect work-pool preview 'pliu-pool' --hours 12

# if a workflow is scheduled on this work pool, you should see something like
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Scheduled Start Time             ┃ Run ID                               ┃ Name                ┃ Deployment ID                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2025-10-29 16:37:52.950000+00:00 │ 019a309e-c009-7080-a931-7f261a479b2a │ archetypal-crayfish │ f950cdcb-1f39-4380-b114-8656322ef64e │
│ 2025-10-29 17:37:52.950000+00:00 │ 019a309e-c009-7793-af29-c0a9131d63c5 │ petite-nightingale  │ f950cdcb-1f39-4380-b114-8656322ef64e │
│ 2025-10-29 18:37:52.950000+00:00 │ 019a309e-c009-7717-b979-4b849dd57fcf │ curly-bullmastiff   │ f950cdcb-1f39-4380-b114-8656322ef64e │
└──────────────────────────────────┴──────────────────────────────────────┴─────────────────────┴──────────────────────────────────────┘

# control the work pool lifecycle
prefect work-pool pause|resume|delete 'pliu-pool'
```

## Concurrency

By default, a Prefect work pool has no concurrency limit
